In [ ]:
!pip install -qq dgl-cu110 dglgo -f https://data.dgl.ai/wheels/repo.html &>/dev/null

In [ ]:
import pandas as pd
import numpy as np
import cudf
from pandas import Timedelta
import os
from tqdm.notebook import tqdm
import pickle
from torch.utils.data import Dataset, DataLoader
import math
from operator import itemgetter
import dgl
pd.set_option('display.max_rows', 10000)

### Dataset
[Last-FM](https://www.kaggle.com/datasets/japarra27/lastfm-dataset)

In [ ]:
data = cudf.read_parquet('/data/items.parquet')
len(data)

In [ ]:
data.head()

In [ ]:
print(f" # Unique User : {data['user_id'].nunique()}, # Unique Artist : {data['artist_id'].nunique()}")